# Probabilistic classification algorithms

In this tutorial we will implement a Naive Bayes classifier for text classification. More specifically, the aim of the classifier is to predict whether a given product review is positive or negative.

We will represent documents (i.e. product reviews) as vectors of binary features. For each [unigram or bigram](https://en.wikipedia.org/wiki/N-gram) that appears in at least one document in our training dataset we introduce a binary feature. For a given document a feature is 1 if the documents contains the corresponding unigram/bigram, and it is 0 otherwise.

The datasets are contained in four files ('test.negative', 'test.positive', 'train.negative', 'train.positive'). The files correspond to product reviews from the test and train datasets labelled positively or negatively. For example, the file 'test.negative' contains negative product reviews from the test dataset.

The reviews are preprocessed and expressed as a list of unigrams and bigrams without duplications (for a bigram (w1,w2), the corresponding record is w1__w2). Each review is represented as a single line and features (unigrams and bigrams) extracted from that review are listed space delimited. Take a moment to inspect the content of the dataset files.

这段文字介绍了一个使用朴素贝叶斯分类器进行文本分类的教程。具体来说，这个分类器的目的是预测给定的产品评论是积极的还是消极的。

在这个教程中，文档（即产品评论）被表示为二元特征的向量。对于训练数据集中至少出现在一个文档里的每个[单词或双词组](https://en.wikipedia.org/wiki/N-gram)，引入一个二元特征。对于给定的文档，如果文档包含相应的单词或双词组，则该特征为1；否则，为0。

数据集包含在四个文件（'test.negative', 'test.positive', 'train.negative', 'train.positive'）中。这些文件对应于测试数据集和训练数据集中标记为积极或消极的产品评论。例如，文件'test.negative'包含测试数据集中的消极产品评论。

评论已经被预处理，并表示为不重复的单词和双词组列表（对于双词组(w1,w2)，相应的记录是w1__w2）。每条评论表示为单独一行，从该评论中提取的特征（单词和双词组）以空格分隔列出。花一点时间查看数据集文件的内容。

In [13]:
import numpy as np

1. First, we read the training data, compute feature statistics, and store it in a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) $\texttt{featureStat}$, where $\texttt{featureStat[feature][classLabel]}$ is equal to the number of documents in the class $\texttt{classLabel}$ that contain $\texttt{feature}$. We also compute the total number of positive train instances and the total number of negative train instances.

这段文字描述了朴素贝叶斯分类器训练过程的第一步。在这一步中，我们需要执行以下操作：

1. **读取训练数据**：加载包含积极和消极产品评论的训练数据集。

2. **计算特征统计**：对于训练集中的每个特征（如单词和双词组），计算它们在积极类别和消极类别中出现的文档数量。

3. **存储特征统计**：将计算得到的特征统计信息存储在一个[字典](https://docs.python.org/3/tutorial/datastructures.html#dictionaries)结构中，称为 `featureStat`。在这个字典中，`featureStat[feature][classLabel]` 的值等于包含特征 `feature` 的类别 `classLabel` 的文档数量。

4. **计算积极和消极实例的总数**：计算训练集中积极类别文档的总数以及消极类别文档的总数。

这些步骤为分类器的训练建立了基础，通过统计每个特征在不同类别文档中出现的次数，分类器能够学习如何区分积极和消极的评论。

In [14]:
def count(featureStat, fname, classLabel):
    numInstances = 0
    with open(fname) as file:
        for line in file:
            numInstances += 1
            for feature in line.strip().split():
                if feature not in featureStat:
                    featureStat[feature] = {0:0, 1:0}
                featureStat[feature][classLabel] += 1
    return numInstances

这段Python代码定义了一个名为count的函数，它用于统计在文本文件中每个特征在每个类别下出现的次数。

函数接受三个参数：

- featureStat：一个字典，用于存储每个特征在每个类别下的出现次数。
- fname：一个字符串，表示要读取的文本文件的名称。
- classLabel：一个整数，表示类别标签（0或1）。
函数的工作流程如下：

1. 初始化一个名为numInstances的变量，用于记录文本文件中的行数（即实例数）。
   
2. 使用with open(fname) as file:打开文本文件。
   
3. 对文件中的每一行执行以下操作：
    1. 使用strip()方法去除行尾的换行符，然后使用split()方法将行分割成特征，得到一个特征列表。
   
    2. 对特征列表中的每个特征执行以下操作：

        1. 如果该特征不在featureStat字典中，则在字典中添加该特征，并将其在两个类别下的出现次数都初始化为0。

        2. 在featureStat字典中找到该特征，然后将其在classLabel类别下的出现次数加1。
   
4. 返回numInstances，即文本文件中的行数。

这个函数可以用于统计文本分类问题中每个特征在每个类别下的出现次数，这对于训练一些概率分类算法（如朴素贝叶斯分类器）是非常有用的。

In [18]:
featureStat = {}
numPositiveInstances = count(featureStat, "train.positive", 1)
numNegativeInstances = count(featureStat, "train.negative", 0)

# 打印前五个
for feature in list(featureStat.keys())[:5]:
    print(feature, featureStat[feature])

atmosphere {0: 1, 1: 1}
they {0: 227, 1: 217}
you__unfamiliar {0: 0, 1: 1}
distant {0: 0, 1: 1}
background__of {0: 0, 1: 1}


2. Now, compute the conditional probabilities $P(w_i = 1 | C = c)$, i.e. probability of specific feature $w_i$ to be present in a document from class $c$. Use Laplace smooting to avoid zero probabilities.

In [17]:
def compute_probabilities(featureStat, numPositiveInstances, numNegativeInstances):
    # 为每个类别创建一个字典来存储特征的条件概率
    probPositive = {}
    probNegative = {}
    
    # 遍历所有的特征
    for feature in featureStat.keys():
        # 应用拉普拉斯平滑，计算特征的条件概率
        probPositive[feature] = (featureStat[feature][1] + 1) / (numPositiveInstances + 2) # +2是因为每个类别的特征计数加了1，同时在分母中加了2（对于二元特征，分母中加的数应该是特征可能取值的数量）
        probNegative[feature] = (featureStat[feature][0] + 1) / (numNegativeInstances + 2)
    
    return probPositive, probNegative

probPositive, probNegative = compute_probabilities(featureStat, numPositiveInstances, numNegativeInstances)
# print("Probilitives of positive class:", probPositive)
# print("Probilitives of negative class:", probNegative)

要计算条件概率 $P(w_i = 1 | C = c)$ 并使用拉普拉斯平滑，你可以按照以下步骤编写代码：

1. 计算每个特征在每个类别中出现的次数，这一步已经在你提供的 count 函数中完成了。

2. 应用拉普拉斯平滑：为了避免出现零概率，对于每个类别，对每个特征的计数加1（即拉普拉斯平滑项）。

3. 计算每个特征的条件概率：将特征在类别中出现的次数除以该类别中文档的总数，再加上拉普拉斯平滑项。

3. Implement a Naive Bayes classfier function that predicts whether a given document belongs to the positive or the negative class. To avoid problems of very small numbers, instead of computing $P(C=c) \cdot \prod_i P(w_i = a_i| C=c)$ cosider computing the $\log$ of that.

这段文字指导你实现一个朴素贝叶斯分类器函数，该函数预测给定文档属于积极类别还是消极类别。由于直接计算概率乘积可能导致非常小的数，可能会遇到数值下溢的问题，所以推荐使用概率的对数来进行计算。这样，乘法操作就转变为加法操作，这在数值计算上更为稳定。

具体地，对于每个类别 \( c \)（在本例中为积极或消极），你将计算对数似然 \( \log P(C=c) \) 加上每个特征 \( w_i \) 的对数条件概率 \( \log P(w_i = a_i| C=c) \) 的总和，其中 \( a_i \) 表示特征 \( w_i \) 在文档中的值（1或0）。在计算中，你还要考虑先验概率 \( P(C=c) \)。

请注意，这个函数的实际实现需要具体的概率值和数据，而且还需要处理诸如数值稳定性和未在训练数据中出现的特征等问题。在计算对数似然时，通常需要考虑所有特征，包括在文档中出现和未出现的。对于未出现的特征，我们使用 \(1 - P(w_i = 1 | C=c)\) 来计算它们对似然的贡献。这里为了简单起见，我们假设每个特征都有一个默认的最小概率。在实际应用中，这可能需要根据特定的数据集进行调整。

In [25]:
import math

def naive_bayes_classifier(doc, numDocuments, probPositive, probNegative, numPositiveInstances, numNegativeInstances):
    # 计算每个类别先验概率的对数
    log_ProbPositive = math.log(numPositiveInstances / numDocuments)
    log_ProbNegative = math.log(numNegativeInstances / numDocuments)
    
    # 初始化 文档属于每个类别的对数 似然为先验概率的对数
    log_likelihood_positive = log_ProbPositive
    log_likelihood_negative = log_ProbNegative
    
    # 遍历文档中的每个特征
    for feature in doc:
        # 如果特征在概率字典中, 累加其对数条件概率
        if feature in probPositive:
            log_likelihood_positive += math.log(probPositive[feature])
        if feature in probNegative:
            log_likelihood_negative += math.log(probNegative[feature])
        # 如果特征不在概率字典中, 使用拉普拉斯平滑后的对数概率
        else:
            log_likelihood_positive += math.log(1 / (numPositiveInstances + 2))
            log_likelihood_negative += math.log(1 / (numNegativeInstances + 2))
    
    # 对于 文档中不存在的特征, 也要考虑它们对似然的贡献
    for frature in featureStat.keys():
        if feature not in doc:
            log_likelihood_positive += math.log(1 - probPositive[feature])
            log_likelihood_negative += math.log(1 - probNegative[feature])
    
    # 比较两个类别的似然, 返回概率较大的类别
    if log_likelihood_positive > log_likelihood_negative:
        return 1
    else:
        return 0
    
# 例子
# newDoc = ['bad', 'bad', 'good']
# numDocuments = numPositiveInstances + numNegativeInstances
# predicted_class = naive_bayes_classifier(newDoc, numDocuments, probPositive, probNegative, numPositiveInstances, numNegativeInstances)
# print("Predicted class:", predicted_class)

4. Let's now read the test dataset from the files

In [19]:
def getInstances(fname, classLabel):
    data = []
    with open(fname) as file:
        for line in file:
            data.append((classLabel, line.strip().split()))
    return data

# Read test data
test_data = getInstances("test.positive", 1)
test_data.extend(getInstances("test.negative", 0))

4. Evaluate accuracy of the Naive Bayes algorithm on the test data

In [27]:
def evaluate_accurancy(test_data, numDocuments, probPositive, probNegative, numPositiveInstances, numNegativeInstances):
    correct_predictions = 0
    
    # 遍历测试数据
    for classLabel, doc in test_data:
        # 使用朴素贝叶斯分类器预测类别
        predicted_class = naive_bayes_classifier(doc, numDocuments, probPositive, probNegative, numPositiveInstances, numNegativeInstances)
        # 如果预测正确, 计数器加一
        if classLabel == predicted_class:
            correct_predictions += 1
    
    # 计算准确率
    accuracy = correct_predictions / len(test_data)
    return accuracy

# 计算test data 准确率
accurancy = evaluate_accurancy(test_data, numPositiveInstances + numNegativeInstances, probPositive, probNegative, numPositiveInstances, numNegativeInstances)
print("Accurancy:", accurancy)

Accurancy: 0.8417085427135679


5. Modify the code and see what happens if we do not use Laplace smoothing